In [77]:
import pandas as pd
import numpy as np
import json
import os
import random
import string
import re
import matplotlib.pyplot as plt

from pathlib import Path
from tqdm import tqdm

import lightgbm as lgb
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error,mean_squared_log_error

pd.set_option('display.max_columns', 500)


In [86]:

df_train = pd.read_pickle('../data/train/full_comp.pickle')
df_test = pd.read_pickle('../data/test/full.pickle')


In [87]:
df_train.head()

,id,submitter,authors,title,comments,journal-ref,doi,report-no,categories,license,abstract,versions,update_date_x,authors_parsed,doi_cites,cites,doi_id,pub_publisher,pub_journals,pub_dois,update_date_y,first_created_date,last_created_date,update_year,first_created_year,last_created_year,update_month,first_created_month,last_created_month,update_ym,first_created_ym,last_created_ym,update_day,first_created_day,last_created_day,update_date_unixtime,first_created_unixtime,last_created_unixtime,diff_update_date_unixtime,diff_created_unixtime,num_created,update_date_days,first_created_days,last_created_days,diff_created_days,author_first,author_num,astro-ph.co,astro-ph.ep,astro-ph.ga,astro-ph.he,astro-ph.im,astro-ph.sr,cond-mat.dis-nn,cond-mat.mes-hall,cond-mat.mtrl-sci,cond-mat.other,cond-mat.quant-gas,cond-mat.soft,cond-mat.stat-mech,cond-mat.str-el,cond-mat.supr-con,cs.ai,cs.ar,cs.cc,cs.ce,cs.cg,cs.cl,cs.cr,cs.cv,cs.cy,cs.db,cs.dc,cs.dl,cs.dm,cs.ds,cs.et,cs.fl,cs.gl,cs.gr,cs.gt,cs.hc,cs.ir,cs.it,cs.lg,cs.lo,cs.ma,cs.mm,cs.ms,cs.na,cs.ne,cs.ni,cs.oh,cs.os,cs.pf,cs.pl,cs.ro,cs.sc,cs.sd,cs.se,cs.si,cs.sy,econ.em,econ.gn,econ.th,eess.as,eess.iv,eess.sp,eess.sy,math.ac,math.ag,math.ap,math.at,math.ca,math.co,math.ct,math.cv,math.dg,math.ds,math.fa,math.gm,math.gn,math.gr,math.gt,math.ho,math.it,math.kt,math.lo,math.mg,math.mp,math.na,math.nt,math.oa,math.oc,math.pr,math.qa,math.ra,math.rt,math.sg,math.sp,math.st,nlin.ao,nlin.cd,nlin.cg,nlin.ps,nlin.si,physics.acc-ph,physics.ao-ph,physics.app-ph,physics.atm-clus,physics.atom-ph,physics.bio-ph,physics.chem-ph,physics.class-ph,physics.comp-ph,physics.data-an,physics.ed-ph,physics.flu-dyn,physics.gen-ph,physics.geo-ph,physics.hist-ph,physics.ins-det,physics.med-ph,physics.optics,physics.plasm-ph,physics.pop-ph,physics.soc-ph,physics.space-ph,q-bio.bm,q-bio.cb,q-bio.gn,q-bio.mn,q-bio.nc,q-bio.ot,q-bio.pe,q-bio.qm,q-bio.sc,q-bio.to,q-fin.cp,q-fin.ec,q-fin.gn,q-fin.mf,q-fin.pm,q-fin.pr,q-fin.rm,q-fin.st,q-fin.tr,stat.ap,stat.co,stat.me,stat.ml,stat.ot,stat.th,acc-phys,adap-org,alg-geom,ao-sci,astro-ph,atom-ph,bayes-an,chao-dyn,chem-ph,cmp-lg,comp-gas,cond-mat,dg-ga,funct-an,gr-qc,hep-ex,hep-lat,hep-ph,hep-th,math-ph,mtrl-th,nucl-ex,nucl-th,patt-sol,plasm-ph,q-alg,q-bio,q-fin,quant-ph,solv-int,supr-con,acc,adap,alg,ao,astro,atom,bayes,chao,chem,cmp,comp,cond,cs,dg,econ,eess,funct,gr,hep,math,mtrl,nlin,nucl,patt,physics,plasm,q,quant,solv,stat,supr,submitter_label,doi_id_label,author_first_label,pub_publisher_label,license_label,doi_cites_mean_author_first_label,doi_cites_count_author_first_label,doi_cites_sum_author_first_label,doi_cites_min_author_first_label,doi_cites_max_author_first_label,doi_cites_median_author_first_label,doi_cites_std_author_first_label,doi_cites_q10_author_first_label,doi_cites_q25_author_first_label,doi_cites_q75_author_first_label,doi_cites_mean_doi_id_label,doi_cites_count_doi_id_label,doi_cites_sum_doi_id_label,doi_cites_min_doi_id_label,doi_cites_max_doi_id_label,doi_cites_median_doi_id_label,doi_cites_std_doi_id_label,doi_cites_q10_doi_id_label,doi_cites_q25_doi_id_label,doi_cites_q75_doi_id_label,doi_cites_mean_pub_publisher_label,doi_cites_count_pub_publisher_label,doi_cites_sum_pub_publisher_label,doi_cites_min_pub_publisher_label,doi_cites_max_pub_publisher_label,doi_cites_median_pub_publisher_label,doi_cites_std_pub_publisher_label,doi_cites_q10_pub_publisher_label,doi_cites_q25_pub_publisher_label,doi_cites_q75_pub_publisher_label,doi_cites_mean_submitter_label,doi_cites_count_submitter_label,doi_cites_sum_submitter_label,doi_cites_min_submitter_label,doi_cites_max_submitter_label,doi_cites_median_submitter_label,doi_cites_std_submitter_label,doi_cites_q10_submitter_label,doi_cites_q25_submitter_label,doi_cites_q75_submitter_label,doi_cites_mean_update_ym,doi_cites_count_update_ym,doi_cites_sum_update_ym,doi_cites_min_update_ym,doi_cites_max_update_ym,doi_cites_median_update_ym,doi_cites_std_update_ym,doi_cites_q10_update_ym,doi_cites_q25_update_ym,doi_cit

In [88]:
df_train = pd.merge(df_train, pd.read_pickle('../data/train/roberta_vec_sample_doi_cites.pickle'), on='id', how='left')

In [89]:
df_train = df_train[df_train['cites'].isnull() == False].reset_index(drop=True)
df_train['cites'] = np.log1p(df_train['cites'])
df_train.shape

(15117, 1101)

In [90]:
df_train.head()

,id,submitter,authors,title,comments,journal-ref,doi,report-no,categories,license,abstract,versions,update_date_x,authors_parsed,doi_cites,cites,doi_id,pub_publisher,pub_journals,pub_dois,update_date_y,first_created_date,last_created_date,update_year,first_created_year,last_created_year,update_month,first_created_month,last_created_month,update_ym,first_created_ym,last_created_ym,update_day,first_created_day,last_created_day,update_date_unixtime,first_created_unixtime,last_created_unixtime,diff_update_date_unixtime,diff_created_unixtime,num_created,update_date_days,first_created_days,last_created_days,diff_created_days,author_first,author_num,astro-ph.co,astro-ph.ep,astro-ph.ga,astro-ph.he,astro-ph.im,astro-ph.sr,cond-mat.dis-nn,cond-mat.mes-hall,cond-mat.mtrl-sci,cond-mat.other,cond-mat.quant-gas,cond-mat.soft,cond-mat.stat-mech,cond-mat.str-el,cond-mat.supr-con,cs.ai,cs.ar,cs.cc,cs.ce,cs.cg,cs.cl,cs.cr,cs.cv,cs.cy,cs.db,cs.dc,cs.dl,cs.dm,cs.ds,cs.et,cs.fl,cs.gl,cs.gr,cs.gt,cs.hc,cs.ir,cs.it,cs.lg,cs.lo,cs.ma,cs.mm,cs.ms,cs.na,cs.ne,cs.ni,cs.oh,cs.os,cs.pf,cs.pl,cs.ro,cs.sc,cs.sd,cs.se,cs.si,cs.sy,econ.em,econ.gn,econ.th,eess.as,eess.iv,eess.sp,eess.sy,math.ac,math.ag,math.ap,math.at,math.ca,math.co,math.ct,math.cv,math.dg,math.ds,math.fa,math.gm,math.gn,math.gr,math.gt,math.ho,math.it,math.kt,math.lo,math.mg,math.mp,math.na,math.nt,math.oa,math.oc,math.pr,math.qa,math.ra,math.rt,math.sg,math.sp,math.st,nlin.ao,nlin.cd,nlin.cg,nlin.ps,nlin.si,physics.acc-ph,physics.ao-ph,physics.app-ph,physics.atm-clus,physics.atom-ph,physics.bio-ph,physics.chem-ph,physics.class-ph,physics.comp-ph,physics.data-an,physics.ed-ph,physics.flu-dyn,physics.gen-ph,physics.geo-ph,physics.hist-ph,physics.ins-det,physics.med-ph,physics.optics,physics.plasm-ph,physics.pop-ph,physics.soc-ph,physics.space-ph,q-bio.bm,q-bio.cb,q-bio.gn,q-bio.mn,q-bio.nc,q-bio.ot,q-bio.pe,q-bio.qm,q-bio.sc,q-bio.to,q-fin.cp,q-fin.ec,q-fin.gn,q-fin.mf,q-fin.pm,q-fin.pr,q-fin.rm,q-fin.st,q-fin.tr,stat.ap,stat.co,stat.me,stat.ml,stat.ot,stat.th,acc-phys,adap-org,alg-geom,ao-sci,astro-ph,atom-ph,bayes-an,chao-dyn,chem-ph,cmp-lg,comp-gas,cond-mat,dg-ga,funct-an,gr-qc,hep-ex,hep-lat,hep-ph,hep-th,math-ph,mtrl-th,nucl-ex,nucl-th,patt-sol,plasm-ph,q-alg,q-bio,q-fin,quant-ph,solv-int,supr-con,acc,adap,alg,ao,astro,atom,bayes,chao,chem,cmp,comp,cond,cs,dg,econ,eess,funct,gr,hep,math,mtrl,nlin,nucl,patt,physics,plasm,...,rbert_vec_raw_768_519,rbert_vec_raw_768_520,rbert_vec_raw_768_521,rbert_vec_raw_768_522,rbert_vec_raw_768_523,rbert_vec_raw_768_524,rbert_vec_raw_768_525,rbert_vec_raw_768_526,rbert_vec_raw_768_527,rbert_vec_raw_768_528,rbert_vec_raw_768_529,rbert_vec_raw_768_530,rbert_vec_raw_768_531,rbert_vec_raw_768_532,rbert_vec_raw_768_533,rbert_vec_raw_768_534,rbert_vec_raw_768_535,rbert_vec_raw_768_536,rbert_vec_raw_768_537,rbert_vec_raw_768_538,rbert_vec_raw_768_539,rbert_vec_raw_768_540,rbert_vec_raw_768_541,rbert_vec_raw_768_542,rbert_vec_raw_768_543,rbert_vec_raw_768_544,rbert_vec_raw_768_545,rbert_vec_raw_768_546,rbert_vec_raw_768_547,rbert_vec_raw_768_548,rbert_vec_raw_768_549,rbert_vec_raw_768_550,rbert_vec_raw_768_551,rbert_vec_raw_768_552,rbert_vec_raw_768_553,rbert_vec_raw_768_554,rbert_vec_raw_768_555,rbert_vec_raw_768_556,rbert_vec_raw_768_557,rbert_vec_raw_768_558,rbert_vec_raw_768_559,rbert_vec_raw_768_560,rbert_vec_raw_768_561,rbert_vec_raw_768_562,rbert_vec_raw_768_563,rbert_vec_raw_768_564,rbert_vec_raw_768_565,rbert_vec_raw_768_566,rbert_vec_raw_768_567,rbert_vec_raw_768_568,rbert_vec_raw_768_569,rbert_vec_raw_768_570,rbert_vec_raw_768_571,rbert_vec_raw_768_572,rbert_vec_raw_768_573,rbert_vec_raw_768_574,rbert_vec_raw_768_575,rbert_vec_raw_768_576,rbert_vec_raw_768_577,rbert_vec_raw_768_578,rbert_vec_raw_768_579,rbert_vec_raw_768_580,rbert_vec_raw_768_581,rbert_vec_raw_768_582,rbert_vec_raw_768_583,rbert_vec_raw_768_584,rbert_vec_raw_768_585,rbert_vec_raw_768_586,rbert_vec_raw_768_587,rbert_vec_raw_768_588,rbert_vec_raw_768_589,rbert_vec_raw_768_590,rbert_vec_

In [107]:
1 / 6 

0.16666666666666666

In [108]:
NFOLDS = 10
kf = KFold(n_splits=NFOLDS, shuffle=True, random_state=777)

df_train['fold_no'] = [0 for x in range(len(df_train))]
for fold, (train_idx, val_idx) in enumerate(kf.split(df_train)):
    df_train.loc[val_idx, 'fold_no'] = fold
df_train.head()


,id,submitter,authors,title,comments,journal-ref,doi,report-no,categories,license,abstract,versions,update_date_x,authors_parsed,doi_cites,cites,doi_id,pub_publisher,pub_journals,pub_dois,update_date_y,first_created_date,last_created_date,update_year,first_created_year,last_created_year,update_month,first_created_month,last_created_month,update_ym,first_created_ym,last_created_ym,update_day,first_created_day,last_created_day,update_date_unixtime,first_created_unixtime,last_created_unixtime,diff_update_date_unixtime,diff_created_unixtime,num_created,update_date_days,first_created_days,last_created_days,diff_created_days,author_first,author_num,astro-ph.co,astro-ph.ep,astro-ph.ga,astro-ph.he,astro-ph.im,astro-ph.sr,cond-mat.dis-nn,cond-mat.mes-hall,cond-mat.mtrl-sci,cond-mat.other,cond-mat.quant-gas,cond-mat.soft,cond-mat.stat-mech,cond-mat.str-el,cond-mat.supr-con,cs.ai,cs.ar,cs.cc,cs.ce,cs.cg,cs.cl,cs.cr,cs.cv,cs.cy,cs.db,cs.dc,cs.dl,cs.dm,cs.ds,cs.et,cs.fl,cs.gl,cs.gr,cs.gt,cs.hc,cs.ir,cs.it,cs.lg,cs.lo,cs.ma,cs.mm,cs.ms,cs.na,cs.ne,cs.ni,cs.oh,cs.os,cs.pf,cs.pl,cs.ro,cs.sc,cs.sd,cs.se,cs.si,cs.sy,econ.em,econ.gn,econ.th,eess.as,eess.iv,eess.sp,eess.sy,math.ac,math.ag,math.ap,math.at,math.ca,math.co,math.ct,math.cv,math.dg,math.ds,math.fa,math.gm,math.gn,math.gr,math.gt,math.ho,math.it,math.kt,math.lo,math.mg,math.mp,math.na,math.nt,math.oa,math.oc,math.pr,math.qa,math.ra,math.rt,math.sg,math.sp,math.st,nlin.ao,nlin.cd,nlin.cg,nlin.ps,nlin.si,physics.acc-ph,physics.ao-ph,physics.app-ph,physics.atm-clus,physics.atom-ph,physics.bio-ph,physics.chem-ph,physics.class-ph,physics.comp-ph,physics.data-an,physics.ed-ph,physics.flu-dyn,physics.gen-ph,physics.geo-ph,physics.hist-ph,physics.ins-det,physics.med-ph,physics.optics,physics.plasm-ph,physics.pop-ph,physics.soc-ph,physics.space-ph,q-bio.bm,q-bio.cb,q-bio.gn,q-bio.mn,q-bio.nc,q-bio.ot,q-bio.pe,q-bio.qm,q-bio.sc,q-bio.to,q-fin.cp,q-fin.ec,q-fin.gn,q-fin.mf,q-fin.pm,q-fin.pr,q-fin.rm,q-fin.st,q-fin.tr,stat.ap,stat.co,stat.me,stat.ml,stat.ot,stat.th,acc-phys,adap-org,alg-geom,ao-sci,astro-ph,atom-ph,bayes-an,chao-dyn,chem-ph,cmp-lg,comp-gas,cond-mat,dg-ga,funct-an,gr-qc,hep-ex,hep-lat,hep-ph,hep-th,math-ph,mtrl-th,nucl-ex,nucl-th,patt-sol,plasm-ph,q-alg,q-bio,q-fin,quant-ph,solv-int,supr-con,acc,adap,alg,ao,astro,atom,bayes,chao,chem,cmp,comp,cond,cs,dg,econ,eess,funct,gr,hep,math,mtrl,nlin,nucl,patt,physics,plasm,...,rbert_vec_raw_768_520,rbert_vec_raw_768_521,rbert_vec_raw_768_522,rbert_vec_raw_768_523,rbert_vec_raw_768_524,rbert_vec_raw_768_525,rbert_vec_raw_768_526,rbert_vec_raw_768_527,rbert_vec_raw_768_528,rbert_vec_raw_768_529,rbert_vec_raw_768_530,rbert_vec_raw_768_531,rbert_vec_raw_768_532,rbert_vec_raw_768_533,rbert_vec_raw_768_534,rbert_vec_raw_768_535,rbert_vec_raw_768_536,rbert_vec_raw_768_537,rbert_vec_raw_768_538,rbert_vec_raw_768_539,rbert_vec_raw_768_540,rbert_vec_raw_768_541,rbert_vec_raw_768_542,rbert_vec_raw_768_543,rbert_vec_raw_768_544,rbert_vec_raw_768_545,rbert_vec_raw_768_546,rbert_vec_raw_768_547,rbert_vec_raw_768_548,rbert_vec_raw_768_549,rbert_vec_raw_768_550,rbert_vec_raw_768_551,rbert_vec_raw_768_552,rbert_vec_raw_768_553,rbert_vec_raw_768_554,rbert_vec_raw_768_555,rbert_vec_raw_768_556,rbert_vec_raw_768_557,rbert_vec_raw_768_558,rbert_vec_raw_768_559,rbert_vec_raw_768_560,rbert_vec_raw_768_561,rbert_vec_raw_768_562,rbert_vec_raw_768_563,rbert_vec_raw_768_564,rbert_vec_raw_768_565,rbert_vec_raw_768_566,rbert_vec_raw_768_567,rbert_vec_raw_768_568,rbert_vec_raw_768_569,rbert_vec_raw_768_570,rbert_vec_raw_768_571,rbert_vec_raw_768_572,rbert_vec_raw_768_573,rbert_vec_raw_768_574,rbert_vec_raw_768_575,rbert_vec_raw_768_576,rbert_vec_raw_768_577,rbert_vec_raw_768_578,rbert_vec_raw_768_579,rbert_vec_raw_768_580,rbert_vec_raw_768_581,rbert_vec_raw_768_582,rbert_vec_raw_768_583,rbert_vec_raw_768_584,rbert_vec_raw_768_585,rbert_vec_raw_768_586,rbert_vec_raw_768_587,rbert_vec_raw_768_588,rbert_vec_raw_768_589,rbert_vec_raw_768_590,rbert_vec_raw_768_591,rbert_vec_

In [109]:
def make_statics_table(target, key, x_train, x_valid, x_test):
    target_encoding = pd.concat([
        pd.DataFrame(x_train.groupby(key)[target].min()).rename(columns={target:f'te_{key}_{target}_min'}),
        pd.DataFrame(x_train.groupby(key)[target].max()).rename(columns={target:f'te_{key}_{target}_max'}),
        pd.DataFrame(x_train.groupby(key)[target].std()).rename(columns={target:f'te_{key}_{target}_std'}),
        pd.DataFrame(x_train.groupby(key)[target].mean()).rename(columns={target:f'te_{key}_{target}_mean'}),
        pd.DataFrame(x_train.groupby(key)[target].median()).rename(columns={target:f'te_{key}_{target}_median'}),
    ], axis = 1)

    x_train = pd.merge(x_train, target_encoding, on=key, how='left')
    x_valid = pd.merge(x_valid, target_encoding, on=key, how='left')
    x_test = pd.merge(x_test, target_encoding, on=key, how='left')

    x_train[f'diff_doi_cites_{key}'] = x_train['doi_cites'] - x_train[f'te_{key}_{target}_mean']
    x_train[f'rate_doi_cites_{key}'] = x_train['doi_cites'] / x_train[f'te_{key}_{target}_mean']
    x_valid[f'diff_doi_cites_{key}'] = x_valid['doi_cites'] - x_valid[f'te_{key}_{target}_mean']
    x_valid[f'rate_doi_cites_{key}'] = x_valid['doi_cites'] / x_valid[f'te_{key}_{target}_mean']
    x_test[f'diff_doi_cites_{key}'] = x_test['doi_cites'] - x_test[f'te_{key}_{target}_mean']
    x_test[f'rate_doi_cites_{key}'] = x_test['doi_cites'] / x_test[f'te_{key}_{target}_mean']

    return x_train, x_valid, x_test

In [110]:
SEED = 777
# --------------------------------------
# パラメータ定義
# --------------------------------------
lgb_params = {
    'objective': 'root_mean_squared_error',
    'boosting_type': 'gbdt',
    'n_estimators': 50000,
    'learning_rate': 0.1,
}

result_y = []
result_proba = []

for fold_no in range(NFOLDS):
    test_fold_no = fold_no
    valid_fold_no = fold_no + 1
    if valid_fold_no == NFOLDS:
        valid_fold_no = 0

    # train
    train = df_train.copy()
    y_train = train[~train['fold_no'].isin([test_fold_no, valid_fold_no])]['cites'].values
    y_valid = train[train['fold_no'] == valid_fold_no]['cites'].values
    y_test = train[train['fold_no'] == test_fold_no]['cites'].values

    train['doi_cites'] = train['doi_cites'].astype('int')
    train = train.drop(
        ['id', 'authors', 'title', 'comments',
        'journal-ref', 'doi', 'report-no', 'categories', 'license',
        'abstract', 'versions', 'update_date_x', 'authors_parsed', 'pub_publisher',
        'update_date_y', 'first_created_date', 'last_created_date', 'doi_id', 'submitter', 'author_first']
        , axis=1
    )

    x_train = train[~train['fold_no'].isin([test_fold_no, valid_fold_no])]
    x_valid = train[train['fold_no'] == valid_fold_no]
    x_test = train[train['fold_no'] == test_fold_no]

    # target encoding
    target = 'cites'
    key = 'update_ym'
    x_train, x_valid, x_test = make_statics_table(target, key, x_train, x_valid, x_test)

    # drop
    x_train = x_train.drop([target], axis=1)
    x_valid = x_valid.drop([target], axis=1)
    x_test = x_test.drop([target], axis=1)
    
    # LightGBM
    model = lgb.LGBMRegressor(**lgb_params)
    model.fit(x_train, y_train,
                eval_set=(x_valid, y_valid),
                eval_metric='mse',
                verbose=100,
                early_stopping_rounds=50,
                categorical_feature=['submitter_label', 'doi_id_label', 'author_first_label', 'pub_publisher_label', 'license_label']
    )

    fold_result = model.predict(x_test)
    result_y.extend(y_test)
    result_proba.extend(fold_result)
    rmsle = mean_squared_error(y_test, fold_result, squared=False)
    print(f"fold {fold_no} lgb score: {rmsle}")

rmsle = mean_squared_error(result_y, result_proba, squared=False)
print("+-" * 40)
print(f"score: {rmsle}")


/usr/local/lib/python3.8/dist-packages/lightgbm/basic.py:1551: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/usr/local/lib/python3.8/dist-packages/lightgbm/basic.py:1554: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['author_first_label', 'doi_id_label', 'license_label', 'pub_publisher_label', 'submitter_label']
  warnings.warn('categorical_feature in Dataset is overridden.\n'
/usr/local/lib/python3.8/dist-packages/lightgbm/basic.py:1286: UserWarning: Overriding the parameters from Reference Dataset.
  warnings.warn('Overriding the parameters from Reference Dataset.')
/usr/local/lib/python3.8/dist-packages/lightgbm/basic.py:1098: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))
Training until validation scores don't improve for 50 rounds
[100]	valid_0's l2: 0.240059	valid_0's rmse: 0.489958
Early stoppin

In [ ]:
0.5060531287008133

In [46]:
x_train.shape

(12094, 314)

In [94]:
feature = x_train.copy()
df_feature = pd.DataFrame(model.booster_.feature_importance(importance_type='gain'), index=feature.columns, columns=['importance']).sort_values('importance', ascending=False)
df_feature.head(50)


,importance
doi_cites,55098.345316
diff_doi_cites_doi_cites_mean_update_ym,5438.629229
diff_doi_cites_doi_cites_mean_first_created_ym,3868.449931
diff_rate_doi_cites_doi_cites_mean_pub_publisher_label,1839.312286
diff_rate_doi_cites_doi_cites_mean_doi_id_label,797.039809
diff_doi_cites_doi_cites_mean_doi_id_label,704.612108
diff_doi_cites_doi_cites_mean_pub_publisher_label,627.994189
last_created_ym,572.020888
diff_rate_doi_cites_doi_cites_mean_update_ym,537.300189
diff_rate_doi_cites_doi_cites_mean_first_created_ym,472.905601


In [95]:
df_feature.tail(50)


,importance
physics.class-ph,0.0
rbert_vec_raw_768_147,0.0
rbert_vec_raw_768_152,0.0
rbert_vec_raw_768_155,0.0
rbert_vec_raw_768_156,0.0
physics.comp-ph,0.0
rbert_vec_raw_768_158,0.0
rbert_vec_raw_768_162,0.0
physics.acc-ph,0.0
rbert_vec_raw_768_210,0.0
